# Installing tensorflow APi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/Tensorflow.zip

Archive:  /content/drive/MyDrive/Tensorflow.zip
   creating: Tensorflow/images/
  inflating: Tensorflow/images/11g.jpg  
  inflating: Tensorflow/images/11g.xml  
  inflating: Tensorflow/images/12g.jpg  
  inflating: Tensorflow/images/12g.xml  
  inflating: Tensorflow/images/14g.jpg  
  inflating: Tensorflow/images/14g.xml  
  inflating: Tensorflow/images/15g.png  
  inflating: Tensorflow/images/15g.xml  
  inflating: Tensorflow/images/16g.jpg  
  inflating: Tensorflow/images/16g.xml  
  inflating: Tensorflow/images/18g.jpg  
  inflating: Tensorflow/images/18g.xml  
  inflating: Tensorflow/images/19g.jpg  
  inflating: Tensorflow/images/19g.xml  
  inflating: Tensorflow/images/1g.jpg  
  inflating: Tensorflow/images/1g.xml  
  inflating: Tensorflow/images/20210325_132935.jpg  
  inflating: Tensorflow/images/20210325_132935.xml  
  inflating: Tensorflow/images/20210325_132946.jpg  
  inflating: Tensorflow/images/20210325_132946.xml  
  inflating: Tensorflow/images/20210325_133017.jpg  
 

In [3]:
%tensorflow_version 1.x
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/cloud-annotations/models

!pip install cloud-annotations==0.0.4
!pip install tf_slim
!pip install lvis
!pip install --no-deps tensorflowjs==1.4.0

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

pwd = os.getcwd()
os.environ['PYTHONPATH'] += f':{pwd}:{pwd}/slim'

TensorFlow 1.x selected.
Cloning into 'models'...
remote: Enumerating objects: 2282, done.
remote: Counting objects: 100% (2282/2282), done.
remote: Compressing objects: 100% (1930/1930), done.
remote: Total 2282 (delta 523), reused 1239 (delta 326), pack-reused 0
Receiving objects: 100% (2282/2282), 30.58 MiB | 1.43 MiB/s, done.
Resolving deltas: 100% (523/523), done.
     |████████████████████████████████| 352 kB 3.8 MB/s 
     |████████████████████████████████| 56 kB 2.3 MB/s 
/content/models/research


In [4]:
!python object_detection/builders/model_builder_tf1_test.py

Running tests under Python 3.7.12: /usr/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[       OK ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feature1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_model_from_config_w

In [5]:
!mkdir /content/output

In [6]:
NUM_TRAIN_STEPS = 50000
MODEL_TYPE = 'ssd_inception_v2_coco_2018_01_28'
CONFIG_TYPE = 'ssd_inception_v2_coco'

CHECKPOINT_PATH = "/content/Tensorflow/workspace/pre-trained-models"
LABEL_MAP_PATH    = os.path.join("/content/Tensorflow/workspace/annotations", 'label_map.pbtxt')
TRAIN_RECORD_PATH = os.path.join("/content/Tensorflow/workspace/annotations", 'train.record')
VAL_RECORD_PATH   = os.path.join("/content/Tensorflow/workspace/annotations", 'test.record')

# Downloading pretrained files

In [7]:
!mkdir {CHECKPOINT_PATH}

In [8]:
Data_PATH = "/content/Tensorflow/workspace/annotations"

In [9]:
import os
import tarfile

import six.moves.urllib as urllib

download_base = 'http://download.tensorflow.org/models/object_detection/'
model = MODEL_TYPE + '.tar.gz'
tmp = '/content/checkpoint.tar.gz'


In [10]:
%cd {CHECKPOINT_PATH}

/content/Tensorflow/workspace/pre-trained-models


In [11]:
!wget {download_base+model}

--2021-09-23 05:20:54--  http://download.tensorflow.org/models/object_detection/ssd_inception_v2_coco_2018_01_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 108.177.97.128, 2404:6800:4008:c00::80
Connecting to download.tensorflow.org (download.tensorflow.org)|108.177.97.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 278114232 (265M) [application/x-tar]
Saving to: ‘ssd_inception_v2_coco_2018_01_28.tar.gz’

ssd_inception_v2_co 100%[===================>] 265.23M  89.8MB/s    in 3.0s    

2021-09-23 05:20:57 (89.8 MB/s) - ‘ssd_inception_v2_coco_2018_01_28.tar.gz’ saved [278114232/278114232]



In [12]:
with tarfile.open("/content/Tensorflow/workspace/pre-trained-models/ssd_inception_v2_coco_2018_01_28.tar.gz") as tar:
    for member in tar.getmembers():
      member.name = os.path.basename(member.name)
      if 'model.ckpt' in member.name:
        tar.extract(member, path=CHECKPOINT_PATH)

In [13]:
%cd /content/models/research

/content/models/research


In [14]:
import re

from google.protobuf import text_format

from object_detection.utils import config_util
from object_detection.utils import label_map_util

pipeline_skeleton = '/content/models/research/object_detection/samples/configs/' + CONFIG_TYPE + '.config'
configs = config_util.get_configs_from_pipeline_file(pipeline_skeleton)

label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)
num_classes = len(label_map.keys())
meta_arch = configs["model"].WhichOneof("model")

override_dict = {
  'model.{}.num_classes'.format(meta_arch): num_classes,
  'train_config.batch_size': 24,
  'train_input_path': TRAIN_RECORD_PATH,
  'eval_input_path': VAL_RECORD_PATH,
  'train_config.fine_tune_checkpoint': os.path.join(CHECKPOINT_PATH, 'model.ckpt'),
  'label_map_path': LABEL_MAP_PATH
}

configs = config_util.merge_external_params_with_configs(configs, kwargs_dict=override_dict)
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_config, "/content/Tensorflow/workspace/annotations")

INFO:tensorflow:Maybe overwriting model.ssd.num_classes: 4
INFO:tensorflow:Maybe overwriting train_config.batch_size: 24
INFO:tensorflow:Maybe overwriting train_input_path: /content/Tensorflow/workspace/annotations/train.record
INFO:tensorflow:Maybe overwriting eval_input_path: /content/Tensorflow/workspace/annotations/test.record
INFO:tensorflow:Maybe overwriting train_config.fine_tune_checkpoint: /content/Tensorflow/workspace/pre-trained-models/model.ckpt
INFO:tensorflow:Maybe overwriting label_map_path: /content/Tensorflow/workspace/annotations/label_map.pbtxt
INFO:tensorflow:Writing pipeline config file to /content/Tensorflow/workspace/annotations/pipeline.config


## Training Model

In [ ]:
!python -m object_detection.model_main \
    --pipeline_config_path=/content/Tensorflow/workspace/annotations/pipeline.config \
    --model_dir=/content/Tensorflow/workspace/annotations \
    --num_train_steps=50000 \
    --num_eval_steps=100

W0923 05:21:21.285328 139800067848064 model_lib.py:801] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 50000
I0923 05:21:21.285579 139800067848064 config_util.py:552] Maybe overwriting train_steps: 50000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0923 05:21:21.285740 139800067848064 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0923 05:21:21.285872 139800067848064 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0923 05:21:21.286078 139800067848064 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0923 05:21:21.286295 139800067848064 model_lib.py:817] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_to

## Creating Inference graph

In [ ]:
OUTPUT_PATH = "/content/Tensorflow/workspace/annotations"

In [ ]:
import os
import re

regex = re.compile(r"model\.ckpt-([0-9]+)\.index")
numbers = [int(regex.search(f).group(1)) for f in os.listdir(OUTPUT_PATH) if regex.search(f)]
TRAINED_CHECKPOINT_PREFIX = os.path.join(OUTPUT_PATH, 'model.ckpt-{}'.format(max(numbers)))

print(f'Using {TRAINED_CHECKPOINT_PREFIX}')



Using /content/Tensorflow/workspace/annotations/model.ckpt-0


In [ ]:
!python -m object_detection.export_inference_graph \
  --pipeline_config_path=/content/Tensorflow/workspace/annotations/pipeline.config \
  --trained_checkpoint_prefix=$TRAINED_CHECKPOINT_PREFIX \
  --output_directory=/content/output

Instructions for updating:
Please use `layer.__call__` method instead.
W0922 17:44:21.973821 139739777628032 deprecation.py:323] From /usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:depth of additional conv before box predictor: 0
I0922 17:44:25.370126 139739777628032 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0922 17:44:25.420293 139739777628032 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv before box predictor: 0
I0922 17:44:25.473149 139739777628032 convolutional_box_predictor.py:156] depth of additional conv before box predictor: 0
INFO:tensorflow:depth of additional conv befo

# Prediction

In [ ]:
# Use the captured photo to make predictions
%matplotlib inline

import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image as PImage
import tensorflow as tf
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import label_map_util



In [ ]:
# Load the labels
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH, use_display_name=True)
# Load the model
path_to_frozen_graph = os.path.join("/content/output", '/content/output/frozen_inference_graph.pb')
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(path_to_frozen_graph, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [ ]:
def prediction(image_path,plot=True): 
  with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
      # these are all basically tensorflow variables
      # Definite input and output Tensors for detection_graph
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
      # Each box represents a part of the image where a particular object was detected.
      detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
      detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
      # loading image using PImage library
      image = PImage.open(image_path)
      # the array based representation of the image will be used later in order to prepare the
      # result image with boxes and labels on it.
      (im_width, im_height) = image.size
      image_np = np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      # Actual detection of the image that we pass through the function.
      (boxes, scores, classes, num) = sess.run(
          [detection_boxes, detection_scores, detection_classes, num_detections],
          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
     
      vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          np.squeeze(boxes),
          np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)
  
  return (boxes, scores, classes, num,image_np)

In [ ]:
def error_detection(image_path,true_count,dataf,a):

  thresh = 0.60
  coordinates = []
  # creating a temporary count where we will store our detected objects
  count = {"nut":0,"screw":0,"bracket":0,"other":0}
  (boxes, scores, classes, num,image_np) = prediction(image_path)   #prediction function is updating the count
  
  # looping through the detected objects 
  for i in range(len(scores[0])):
    # only consider it if score is greater than 60%
    if scores[0][i]>=thresh:
      current = boxes[0][i]
      coordinates.append(current)
      # then we will increase the count of that obj in our local count variable
      name = category_index[classes[0][i]]["name"] 
      count[name] = count[name]+1
  
  count1=count.copy()
  dataf.append(count1)    
  print(count)
  

  # c=0 is basically a variable. We will only increase its value if the true count is equal to count for any of the four objects
  # For example: if nut is not the same then c will stay 0, if the value of screw is the same then c will be 1
  #              if bracket is not same then c will stay at 1 and if same then c will be 2 
  c=0
  cc=a
  dd=0
  for obj in count.keys():
    if count[obj] != true_count[obj]:
      diff = true_count[obj]-count[obj]
      if diff>=0:
        print("Error! {} {} is/are missing".format(diff,obj))
        dataf.append("Error! {} {} is/are missing".format(diff,obj))
        count[obj] = count[obj]+diff
             
      else:
        print("Error! {} {} is/are excess".format(diff*-1,obj))
        dataf.append("Error! {} {} is/are excess".format(diff*-1,obj))
        count[obj] = count[obj] - (diff*-1)
      
    else:
      # count=true_count for all the categories
      c=c+1


  coordinates = []
  count = {"nut":0,"screw":0,"bracket":0,"other":0}
  (boxes, scores, classes, num,image_np) = prediction(image_path)
  for i in range(len(scores[0])):
    if scores[0][i]>=thresh:
      current = boxes[0][i]
      coordinates.append(current)
      name = category_index[classes[0][i]]["name"] 
      count[name] = count[name]+1
  for obj in count.keys():
    
    if count[obj] != true_count[obj]:
      
      diff = true_count[obj]-count[obj]
      if obj=="other" and diff!=0:
        print("Remove the other/unknown object and then move to the staging area")
        dataf.append("Remove the other/unknown object")
        break
      if diff>0:
        #print("Error! {} {} are missing".format(diff,obj))
        #count[obj] = count[obj]+diff
        print(" {} {} added to the CURRENT BIN from the PARTS BIN ".format(diff,obj))
        dataf.append(" {} {} added to the CURRENT BIN from the PARTS BIN ".format(diff,obj))
      else: 
        #print("Error! {} {} are excess".format(diff*-1,obj))
        #count[obj] = count[obj] - (diff*-1)
        print(" {} {} removed from the CURRENT BIN to the PARTS BIN".format(diff*-1,obj))
        dataf.append(" {} {} removed from the CURRENT BIN to the PARTS BIN".format(diff*-1,obj))
  
  if c==len(count):
    # Here basically we are checking if the length of the count is 4 and if it is 4 then it means that the
    # true count and count is same for all the 4 parts. In that case we can move to the staging area
    print("Entire order is here, move to the staging area")
    dataf.append("Entire order is here, move to the staging area")
    # cc=a=0, so now cc=0+1=1. Then it will perform image processing again in the next cell
    cc=cc+1

  return (count,image_np,cc,dd) # ignore dd

In [ ]:
def pred_images(im_path,t_count,dataf):
  
  # Checking if its a directory or a single image 
  error = pd.DataFrame(columns = list(t_count.keys()))
  if os.path.isdir(im_path):
  # We go to the else part basically if its a single image. 
  # For both scenarios, it asks the user for the input and then goes to the error detection function 
  # Dataf.append is basically a data frame that will later be converted into a txt form
    images = os.listdir(im_path)
    img_names = []
    for img in images:
      
      try:
        a=0
        b=0
        if img.endswith("jpg"):
          
          img_path = im_path+"/"+img
          print('Name:{}'.format(img))
          dataf.append('Name:{}'.format(img))
          t_count["nut"] = int(input("Enter number of Nuts :: "))
          t_count["screw"] = int(input("Enter number of Screw :: "))
          t_count["bracket"] = int(input("Enter number of Bracket :: "))
          t_count["other"] = int(input("Enter number of Other :: "))
          dataf.append("Nuts needed: {}".format(t_count["nut"]))
          dataf.append("Screws needed: {}".format(t_count["screw"]))
          dataf.append("bracket needed: {}".format(t_count["bracket"]))
          dataf.append("Other needed: {}".format(t_count["other"]))
          count,image_np,c1,d1 = error_detection(img_path,t_count,dataf,a)
          if c1==1: # C1 is a flag to re check if this is 1 then we have the 4th scenario 
            print("Performing image processing again")
            del dataf[-1]
            del dataf[-1]
            del dataf[-1]
            del dataf[-1]
            del dataf[-1]
            del dataf[-1]
            dataf.append(("Performed image processing again"))
            dataf.append("Nuts needed: {}".format(t_count["nut"]))
            dataf.append("Screws needed: {}".format(t_count["screw"]))
            dataf.append("bracket needed: {}".format(t_count["bracket"]))
            dataf.append("Other needed: {}".format(t_count["other"]))       # calling error detection function again for rechecking
            count,image_np,c2,d1 = error_detection(im_path,t_count,dataf,a) # a is initially 0 but if true count and count are the same then a becomes 1 and #d1 is just variable for any further changes
          dataf.append("-------------------------------------------------------------")
      except :
        continue
    error["Image"] = img_names
    #error.to_csv("/content/output/error.txt",index=False)
  else :
    a=0  # this is for the 4th scenario
    b=0  # we don't need b
    print('Name:{}'.format(im_path))
    dataf.append("Name: {}".format(im_path))
    t_count["nut"] = int(input("Enter number of Nuts :: "))
    t_count["screw"] = int(input("Enter number of Screw :: "))
    t_count["bracket"] = int(input("Enter number of Bracket :: "))
    t_count["other"] = int(input("Enter number of Other :: "))
    dataf.append("Nuts needed: {}".format(t_count["nut"]))
    dataf.append("Screws needed: {}".format(t_count["screw"]))
    dataf.append("bracket needed: {}".format(t_count["bracket"]))
    dataf.append("Other needed: {}".format(t_count["other"]))
    count2,image_np,c1,d1 = error_detection(im_path,t_count,dataf,a)
    if c1==1:
            print("\n")
            print("Performing image processing again")
            print("\n")
            del dataf[-1] #deleting wrong values ie values before double check so that we only get the latest values on the txt file
            del dataf[-1]
            del dataf[-1]
            del dataf[-1]
            del dataf[-1]
            del dataf[-1]
            dataf.append(("Performed image processing again"))
            dataf.append("Nuts needed: {}".format(t_count["nut"]))
            dataf.append("Screws needed: {}".format(t_count["screw"]))
            dataf.append("bracket needed: {}".format(t_count["bracket"]))
            dataf.append("Other needed: {}".format(t_count["other"]))
            count,image_np,c3,d1 = error_detection(im_path,t_count,dataf,a) # rechecking, basically performing image processing again
            
    dataf.append("-------------------------------------------------------------")
    plt.figure(figsize=(12, 8))
    plt.imshow(image_np)
  return dataf

In [ ]:
# Storing the old scenarios in dq so everything prints out in the txt file one after another
dq=[]

In [ ]:
true_count = {"nut":0,"screw":0,"bracket":0,"other":0}
 
dataf=dq
# true_count["nut"] = int(input("Enter number of Nuts :: "))
# true_count["screw"] = int(input("Enter number of Screw :: "))
# true_count["bracket"] = int(input("Enter number of Bracket :: "))
# true_count["other"] = int(input("Enter number of Other :: "))

# true_count["nut"] = 0
# true_count["screw"] = 4
# true_count["bracket"] = 0
# true_count["other"] = 0
# dataf.append("Nuts needed: {}".format(true_count["nut"]))
# dataf.append("Screws needed: {}".format(true_count["screw"]))
# dataf.append("bracket needed: {}".format(true_count["bracket"]))
# dataf.append("Other needed: {}".format(true_count["other"]))
img_path = "/content/Tensorflow/images/20210325_135214.jpg"        # giving the input path for the image; can either be a single image or a directory

pred_images(img_path,true_count,dataf)
# Put the data in the df and then output the df in a txt file. Basically we have everything stored in dataf
df = pd.DataFrame(dataf)
# Here dq has the old data and dataf has the new data
dq=dataf
df.to_csv("/content/output/error.txt",index=False) #2 15 62 21 43 16 41 0

Name:/content/Tensorflow/images/20210325_135214.jpg
Enter number of Nuts :: 3
Enter number of Screw :: 3
Enter number of Bracket :: 2
Enter number of Other :: 0
{'nut': 3, 'screw': 3, 'bracket': 2, 'other': 0}
Entire order is here, move to the staging area


Performing image processing again


{'nut': 3, 'screw': 3, 'bracket': 2, 'other': 0}
Entire order is here, move to the staging area
